# Simulation Settings
Define the settings for the MoMaLeNe fluid simulation.

In [3]:
from PIL import Image
from lib.simulator import Simulator
from lib.renderer import Renderer
from lib.video import create_video
from tqdm import tqdm
from pathlib import Path
from datetime import datetime
from lib.timer import TimeTracker
from IPython.display import Video

# Rendering Settings
width = 300
height = 300
fps = 60
total_frames = 500

# Simulator Settings
grid_size = 8
num_particles = grid_size * grid_size
gravity = 3e-04
border = 1.0 
damping_factor = 0.8

# Data Settings
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
root_dir = Path.cwd()
simulation_dir = root_dir / "videos"
video_dir = simulation_dir / timestamp 
video_path = video_dir / "video.mp4"
frame_dir = video_dir / "frames"

# Debug Settings
verbose = True

# Track timings for the simulation and the renderer
time_tracker = TimeTracker()

# Create the Simulator
simulator = Simulator(
    num_particles=num_particles,
    gravity=gravity,
    border=border,
    damping_factor=damping_factor,
)
# Create the Renderer
renderer = Renderer(
    width=width,
    height=height,
)

# Precompute all the frames and store them to disk
for frame_idx in tqdm(range(total_frames)):
    time_tracker.start("simulator")
    state = simulator.step()
    time_tracker.start("renderer", stop=True)
    image = renderer.render(state=state)
    time_tracker.start("frames", stop=True)
    path = frame_dir / f"{frame_idx:05}.png"
    path.parent.mkdir(exist_ok=True, parents=True)
    Image.fromarray(image).save(path)
    time_tracker.stop("frames")

# Create video from precomputed frames and stores the result
time_tracker.start("video")
create_video(
    video_dir=frame_dir,
    video_path=video_path,
    framerate=fps,
)
time_tracker.stop("video")

if verbose:
    print(time_tracker.summary())

# Play the video in jupyter notebook
display(Video(video_path))

100%|██████████| 500/500 [00:04<00:00, 123.59it/s]



+-----------+---------+---------+---------+---------+-------+-------+----------+
| task_name |    mean |     min |     max |  median |   std | steps |    total |
+-----------+---------+---------+---------+---------+-------+-------+----------+
| simulator |   0.096 |   0.077 |   0.834 |   0.089 | 0.047 |   500 |   48.106 |
|  renderer |   4.849 |   4.251 |  31.123 |   4.467 | 1.741 |   500 | 2424.744 |
|    frames |   3.101 |   2.796 |   27.86 |   3.021 | 1.148 |   500 | 1550.417 |
|     video | 2894.69 | 2894.69 | 2894.69 | 2894.69 |   0.0 |     1 |  2894.69 |
+-----------+---------+---------+---------+---------+-------+-------+----------+
